In [1]:
import numpy as np
# import requests

import cv2
import time

import imutils

from directkeys import  W, A, S, D, Left, Right, Up, Down
from directkeys import PressKey, ReleaseKey 

from imutils.video import FileVideoStream, VideoStream

path_vid = 'Material/Videos/full.mp4'
path_vid_small = 'Material/Videos/short.mp4'

# video = VideoStream(src=path_vid).start()
 
# # allow the camera or video file to warm up
# time.sleep(2.0)
# initial = True
# flag = False
# current_key_pressed = set()
# circle_radius = 30

In [2]:
key_up = W
key_down = S
key_left = A
key_right = D


size_height = 600#360
size_width = 600#480
tracker_type = 'CSRT'

In [3]:
KEYS = {17:'W', 30:'A', 31:'S', 32:'D'}

In [5]:
def draw(ret, bbox, frame):
         # Draw bounding box
    if ret:
        # Tracking success
        p1 = (int(bbox[0]), int(bbox[1]))
        p2 = (int(bbox[0] + bbox[2]), int(bbox[1] + bbox[3]))
        cv2.rectangle(frame, p1, p2, (255,0,0), 2, 1)
    else :
        # Tracking failure
        cv2.putText(frame, "Tracking failure detected", (100,80), cv2.FONT_HERSHEY_SIMPLEX, 0.75,(0,0,255),2)


def get_centroid(bbox): #center point of rectangle
    x, y, w, h = bbox
    centx = int(x + w // 2)
    centy = int(y + h // 2)
    return (centx, centy)

def drawbox(ret, bbox, frame): #draws rectangle from bbox
    global q
    if ret:
        #Tracking Success
        p1 = (int(bbox[0]), int(bbox[1]))
        p2 = (int(bbox[0] + bbox[2]), int(bbox[1] + bbox[3]))
        cv2.rectangle(frame, p1, p2, (255,0,0), 2, 1)

    if p1[0] < 10: #quits program if taken hand to left top corner
        q = True # quit

In [6]:
key_steer, key_acc = None, None #key to accelerate
pressed pressed_steer = None, None

# delay = 0.3
last = 0 #last extreme of steer
chng = 3 #change in last, to remove small changes fulucations
chngt = 3 #to restart change to 0, if moved towards centre by some value
# prev = 0 #store last pos of steer, so that when returngin towards centre, doesnt press that key

COLOR_RED = (0, 0, 255)
COLOR_BLACK = (0, 0, 0)

THRESH_ACTION = 40 # threshold for a action to be registered, inside this no movement
# TAS = T #ta steer

def steer(bbox): #to steer left and right
    global last, pressed_steer, key_steer
    x, _ = get_centroid(bbox)
    diff = x - cent[0] 
    
    if abs(diff) < last: #2ns so, new diff is more, 
        if abs(diff) < last-chng*chngt:
            last = 0
        return
    last = abs(diff) + chng
    
    if abs(diff) < THRESH_ACTION:
            return        
    

    if diff > 0:
        key_steer = key_right
    else:
        key_steer = key_left
        
    pressed_steer = True
    PressKey(key_steer)
    return diff    
    
def accelerate(bbox):
    global pressed_acc, key_acc
    
    _, y = get_centroid(bbox)
    diff = - (y - cent[1])  # '-' as y decr upward

    if abs(diff) < THRESH_ACTION:         
        return
    
    if diff > 0:
        key_acc = key_up
    else:
        key_acc = key_down
        
    pressed_acc = True    
    PressKey(key_acc)
    return diff


def draw_circle(frame, center, radius=THRESH_ACTION, color=COLOR_BLACK, thickness=3):
    cv2.circle(frame, center, radius, color, thickness)
 
def release_key(pressed, key):
    if not pressed:
        ReleaseKey(key)
        
        

def action(bbox):
    global pressed_acc, pressed_steer, key_steer, key_acc
    
    pressed_acc = False
    pressed_steer = False
    
    diff_acc = accelerate(bbox) #a
    diff_steer = steer(bbox) #s
        
    if not pressed_acc and key_acc is not None:
        ReleaseKey(key_acc)
        key_acc = None
        
    if not pressed_steer and key_steer is not None:
        ReleaseKey(key_steer)
        key_steer = None
    
    return diff_acc, diff_steer

# cv2.destroyAllWindows()

In [14]:
import requests
import threading

vid = False
url = 'http://192.168.43.1:8080'

def get_frame_phone():
    img_res = requests.get(url+'/shot.jpg')
    img_arr = np.array(bytearray(img_res.content), dtype = np.uint8)
    img = cv2.imdecode(img_arr,-1)
    return img

def get_frame():
    frame = get_frame_phone() # fvs.read()
    if frame is None:
        return
    frame = cv2.flip(frame, 1)
    frame = imutils.resize(frame, width=size_width, height=size_height) 
    return frame

In [16]:
# fvs = FileVideoStream(path_vid).start() #vs Video Stream
# time.sleep(2.0) #to allow web cam to open

TIMER_SETUP = 3. # timer for capturing base image, get reading in posture
# mult = 10
t = time.time()

while True:
    frame = get_frame()
    curr = (time.time() - t)
    if curr > TIMER_SETUP:
        break
    cv2.putText(frame, str(int(TIMER_SETUP - curr)+1), (225,255), cv2.FONT_HERSHEY_SIMPLEX, 1.5, COLOR_RED, 4)
    cv2.imshow("Setup", frame)
    cv2.waitKey(1)

FRAME = frame.copy()
cv2.destroyAllWindows()

#Make box around hand
frame = FRAME.copy()
cv2.putText(frame, 'Select Hand', (30,30), cv2.FONT_HERSHEY_SIMPLEX, 0.75, COLOR_RED, 2)
bbox = cv2.selectROI(frame, False) # bounding box for left hand 

cent = get_centroid(bbox)

draw_circle(frame, cent) # circle, outside this movements will happend

BB = bbox # saving it for later

cv2.destroyAllWindows()

In [52]:
# fvs = FileVideoStream(path_vid).start() #vs Video Stream
bbox = BB

# Creating the CSRT Tracker
tracker = cv2.TrackerCSRT_create() # left hand for steering

# Initialize tracker with first frame and bounding box
tracker.init(FRAME, bbox)

cv2.putText(frame.copy(), 'Put both your hands in Postion', (100,70), \
    cv2.FONT_HERSHEY_SIMPLEX, 0.75, COLOR_BLACK, 2)

TIMER_SETUP = 5
t = time.time()

while True:
    frame = get_frame()
    curr = (time.time() - t)
    if curr > TIMER_SETUP or frame is None:
        break
    cv2.putText(frame, str(int(TIMER_SETUP - curr)+1), (225,255), cv2.FONT_HERSHEY_SIMPLEX, 1.5, COLOR_RED, 4)
    drawbox(True, bbox, frame)
    cv2.imshow("Tracking", frame)
    cv2.waitKey(1)

cv2.destroyAllWindows()


q = False #to quit, take your hand on top corner

while True:
    frame = get_frame()
    
    if frame is None:
        break
        
#     frame = cv2.flip(frame, 1)
#     frame = imutils.resize(frame, width=size_width, height=)  

    ret, bbox = tracker.update(frame) # retleft - True, if found hand
    
    if not ret == True:
        print("Tracking stopped")
        break
    
    diff_acc, diff_steer  = action(bbox)
    
#     cv2.putText(frame, "Acceleration -> " + str(diff_acc), (100,70), cv2.FONT_HERSHEY_SIMPLEX, 0.75, COLOR_BLACK, 2)
#     cv2.putText(frame, "Steer -> " + str(diff_steer), (100,90), cv2.FONT_HERSHEY_SIMPLEX, 0.75, COLOR_BLACK, 2)
    activekeys = ' '.join([KEYS[i] for i in [key_acc, key_steer] if i])
    cv2.putText(frame, "Keys -> " + str(activekeys), (100,90), cv2.FONT_HERSHEY_SIMPLEX, 0.75, COLOR_BLACK, 2)
     
    drawbox(ret, bbox, frame)

    draw_circle(frame, cent)

    cv2.putText(frame, tracker_type + " Tracker", (100,20), cv2.FONT_HERSHEY_SIMPLEX, 0.75, COLOR_BLACK,2);
    cv2.imshow("Tracking", frame)
    
    k = cv2.waitKey(1)

    if k == 13 or q: #13 is the Enter Key
        break
        
ReleaseKey(key_steer)
ReleaseKey(key_acc)

cv2.destroyAllWindows()
fvs.stop()